In [8]:
%%javascript
element.append("<div id='content'></div>");

<IPython.core.display.Javascript object>

In [9]:
%%javascript
requirejs.config({
    paths: {
        T: ['//mrdoob.github.com/three.js/build/three']
    }
});
require(['T'], function(T) {
    window.THREE = T;
});

<IPython.core.display.Javascript object>

In [10]:
%%html
<script src="https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min.map"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>

<script src="https://s3-eu-west-1.amazonaws.com/molab-threejs/OrbitControls.js"></script>
<script src="https://s3-eu-west-1.amazonaws.com/molab-threejs/Detector.js"></script>


In [11]:
%%javascript

(function() {
    
    var fname;
    var lineNum;
    var thisMaxIndex;
    var vars;
    var dx;
    var group;
    
    var parentDiv = document.getElementById("content");
    var rgbToHex = function (rgb) {
	  var hex = Number(rgb).toString(16);
	  if (hex.length < 2) {
	       hex = "0" + hex;
	  }
	  return hex;
	};

	var fullColorHex = function(r,g,b) {
	  var red = rgbToHex(r);
	  var green = rgbToHex(g);
	  var blue = rgbToHex(b);
	  return red+green+blue;
	};

	var camera, scene, renderer;
	var meshes=[];
	var scale=0;
	var didScale=false;
	init();
	animate();
	function init(){
		scene = new THREE.Scene();
		scene.background = new THREE.Color( 0xcccccc );
		renderer = new THREE.WebGLRenderer( { antialias: true } );
		renderer.setPixelRatio( window.devicePixelRatio );
		renderer.setSize( window.innerWidth, window.innerHeight );

		renderer.localClippingEnabled = true;

		document.body.appendChild( renderer.domElement );
		// camera
		camera = new THREE.PerspectiveCamera( 40, window.innerWidth / window.innerHeight, 1, 1000 );
		camera.position.set( 0,0,0 );
		scene.add( camera );
		// controls
		var controls = new THREE.OrbitControls( camera, renderer.domElement );
		controls.minDistance = 1;
		controls.maxDistance = 2;
		controls.maxPolarAngle = 2*Math.PI;
		scene.add( new THREE.AmbientLight( 0xaaaaaa ) );
		// light

		var light = new THREE.PointLight( 0xffffff, .5 );
		camera.add( light );

		//var directionalLight = new THREE.DirectionalLight( 0xffffff, 1 );
		//scene.add( directionalLight );

		// helper
		scene.add( new THREE.AxesHelper( 2 ) );
		group = new THREE.Group();
		scene.add( group );

		var xbar=0;
		var ybar=0;
		var zbar=0;
		var npts=0;
		var xmin=1e8;
		var xmax=-1e8;
		var zmin=1e8;
		var zmax=-1e8;

		var globalMinZ=1e8;
		var globalMaxZ=-1e8;

		for (var ii=0;ii<7;ii++){
		    var i=ii;
		    var verts=[];
		    var tris = [];


		    fname="dataFiles/mayavi/NigeriaNonDying_surf_"+i+"_horizon.txt";
		    if(i==6){
		        fname="dataFiles/mayavi/NigeriaNonDying_fault.txt";
			}
		    //fname='dataFiles/mayavi/smallTest_'+i+'.txt';
		    //fname="dataFiles/mayavi/tester_"+str(i)+".ts";

			var myData;

			jQuery.ajaxSetup({async:false});
			jQuery.get(fname, function(data) {
			   //alert(data);
			   myData=data.split('\n');
			   //console.log("datadata:  ",data);
			   //console.log("mydatadata:  ",myData);

			   //process text file line by line
			   //$('#div').html(data.replace('n',''));
			});
			//console.log("done");

			var pts=[];
			var tris=[];
			myData.forEach(function(d){

			    lineNum = 0
			    thisMaxIndex = 0
		        vars = d.split(' ')
		        type = vars[0]
		        if (type == "VRTX"){
		            var k = +vars[1];
		            var p1 = +(vars[2])
		            var p2 = +(vars[3])
		            var p3 = +(vars[4])-10570.0030146;
		            xmin = Math.min(xmin, p1)
		            xmax = Math.max(xmax, p1)
					globalMinZ=Math.min(globalMinZ,p3)
					globalMaxZ=Math.max(globalMaxZ,p3)
		            if (!didScale){
		                xbar = xbar + p1
		                ybar = ybar + p2
		                zbar = zbar + p3
		                npts = npts + 1
					}
		            verts.push([p1, p2, -p3])
				}

		        if (type == "TRGL"){
		            p1 = +(vars[1]) - 1
		            p2 = +(vars[2]) - 1
		            p3 = +(vars[3]) - 1
		            tris.push([p1, p2, p3])
				}
			})

		    if (!didScale){
		        xbar = xbar / npts;
		        ybar = ybar / npts;
		        zbar = zbar / npts;
		        dx = xmax - xmin;
		        scale = 1 / dx;
				didScale=true;
			}

		    var verts2=[]
		    verts.forEach(function(v){
		        verts2.push([(v[0] - xbar) * scale, (v[1] - ybar) * scale, (v[2] - zbar*0) * scale])
			})

		    console.log("g:  ",xbar*scale, ybar*scale, zbar*scale, (zmin-zbar)*scale, (zmax-zbar)*scale,scale);

			var dz=17043.9657354*scale
			var minZ=(zmin-zbar)*scale;



			var minZ=1e8;
			var maxZ=-1e8;
			verts2.forEach(function(d){
				minZ=Math.min(minZ,d[2])
				maxZ=Math.max(maxZ,d[2])
			})
			dz=maxZ-minZ;
			dz=17043.9657354*scale;

			var mycolorscale=[];
			mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,1,1)");mycolorscale.push("rgb(1,7,8)");mycolorscale.push("rgb(1,7,8)");mycolorscale.push("rgb(1,14,15)");mycolorscale.push("rgb(1,14,15)");mycolorscale.push("rgb(1,20,22)");mycolorscale.push("rgb(1,20,22)");mycolorscale.push("rgb(1,27,29)");mycolorscale.push("rgb(1,27,29)");mycolorscale.push("rgb(1,34,36)");mycolorscale.push("rgb(1,34,36)");mycolorscale.push("rgb(1,40,43)");mycolorscale.push("rgb(1,40,43)");mycolorscale.push("rgb(1,47,50)");mycolorscale.push("rgb(1,47,50)");mycolorscale.push("rgb(1,54,58)");mycolorscale.push("rgb(1,54,58)");mycolorscale.push("rgb(1,60,64)");mycolorscale.push("rgb(1,60,64)");mycolorscale.push("rgb(1,66,71)");mycolorscale.push("rgb(1,66,71)");mycolorscale.push("rgb(1,72,78)");mycolorscale.push("rgb(1,72,78)");mycolorscale.push("rgb(1,79,85)");mycolorscale.push("rgb(1,79,85)");mycolorscale.push("rgb(1,86,92)");mycolorscale.push("rgb(1,86,92)");mycolorscale.push("rgb(1,92,99)");mycolorscale.push("rgb(1,92,99)");mycolorscale.push("rgb(1,99,106)");mycolorscale.push("rgb(1,99,106)");mycolorscale.push("rgb(1,106,114)");mycolorscale.push("rgb(1,106,114)");mycolorscale.push("rgb(1,106,116)");mycolorscale.push("rgb(1,106,116)");mycolorscale.push("rgb(1,107,119)");mycolorscale.push("rgb(1,107,119)");mycolorscale.push("rgb(1,107,122)");mycolorscale.push("rgb(1,107,122)");mycolorscale.push("rgb(1,108,124)");mycolorscale.push("rgb(1,108,124)");mycolorscale.push("rgb(1,108,127)");mycolorscale.push("rgb(1,108,127)");mycolorscale.push("rgb(1,109,130)");mycolorscale.push("rgb(1,109,130)");mycolorscale.push("rgb(1,109,133)");mycolorscale.push("rgb(1,109,133)");mycolorscale.push("rgb(1,110,135)");mycolorscale.push("rgb(1,110,135)");mycolorscale.push("rgb(1,111,138)");mycolorscale.push("rgb(1,111,138)");mycolorscale.push("rgb(1,111,141)");mycolorscale.push("rgb(1,111,141)");mycolorscale.push("rgb(1,112,144)");mycolorscale.push("rgb(1,112,144)");mycolorscale.push("rgb(1,112,146)");mycolorscale.push("rgb(1,112,146)");mycolorscale.push("rgb(1,113,149)");mycolorscale.push("rgb(1,113,149)");mycolorscale.push("rgb(1,113,152)");mycolorscale.push("rgb(1,113,152)");mycolorscale.push("rgb(1,114,155)");mycolorscale.push("rgb(1,114,155)");mycolorscale.push("rgb(1,115,158)");mycolorscale.push("rgb(1,115,158)");mycolorscale.push("rgb(3,114,160)");mycolorscale.push("rgb(3,114,160)");mycolorscale.push("rgb(6,114,163)");mycolorscale.push("rgb(6,114,163)");mycolorscale.push("rgb(9,114,166)");mycolorscale.push("rgb(9,114,166)");mycolorscale.push("rgb(12,114,169)");mycolorscale.push("rgb(12,114,169)");mycolorscale.push("rgb(15,113,172)");mycolorscale.push("rgb(15,113,172)");mycolorscale.push("rgb(18,113,174)");mycolorscale.push("rgb(18,113,174)");mycolorscale.push("rgb(21,113,177)");mycolorscale.push("rgb(21,113,177)");mycolorscale.push("rgb(24,113,180)");mycolorscale.push("rgb(24,113,180)");mycolorscale.push("rgb(27,112,183)");mycolorscale.push("rgb(27,112,183)");mycolorscale.push("rgb(30,112,186)");mycolorscale.push("rgb(30,112,186)");mycolorscale.push("rgb(33,112,188)");mycolorscale.push("rgb(33,112,188)");mycolorscale.push("rgb(36,112,191)");mycolorscale.push("rgb(36,112,191)");mycolorscale.push("rgb(39,111,193)");mycolorscale.push("rgb(39,111,193)");mycolorscale.push("rgb(42,111,196)");mycolorscale.push("rgb(42,111,196)");mycolorscale.push("rgb(45,111,199)");mycolorscale.push("rgb(45,111,199)");mycolorscale.push("rgb(48,111,206)");mycolorscale.push("rgb(48,111,206)");mycolorscale.push("rgb(52,111,209)");mycolorscale.push("rgb(52,111,209)");mycolorscale.push("rgb(56,111,212)");mycolorscale.push("rgb(56,111,212)");mycolorscale.push("rgb(60,111,215)");mycolorscale.push("rgb(60,111,215)");mycolorscale.push("rgb(64,111,219)");mycolorscale.push("rgb(64,111,219)");mycolorscale.push("rgb(67,111,222)");mycolorscale.push("rgb(67,111,222)");mycolorscale.push("rgb(71,111,225)");mycolorscale.push("rgb(71,111,225)");mycolorscale.push("rgb(75,111,228)");mycolorscale.push("rgb(75,111,228)");mycolorscale.push("rgb(79,111,232)");mycolorscale.push("rgb(79,111,232)");mycolorscale.push("rgb(83,111,235)");mycolorscale.push("rgb(83,111,235)");mycolorscale.push("rgb(87,111,238)");mycolorscale.push("rgb(87,111,238)");mycolorscale.push("rgb(91,111,241)");mycolorscale.push("rgb(91,111,241)");mycolorscale.push("rgb(95,111,245)");mycolorscale.push("rgb(95,111,245)");mycolorscale.push("rgb(99,111,248)");mycolorscale.push("rgb(99,111,248)");mycolorscale.push("rgb(103,111,251)");mycolorscale.push("rgb(103,111,251)");mycolorscale.push("rgb(107,111,254)");mycolorscale.push("rgb(107,111,254)");mycolorscale.push("rgb(112,112,254)");mycolorscale.push("rgb(112,112,254)");mycolorscale.push("rgb(114,105,254)");mycolorscale.push("rgb(114,105,254)");mycolorscale.push("rgb(116,98,254)");mycolorscale.push("rgb(116,98,254)");mycolorscale.push("rgb(118,91,254)");mycolorscale.push("rgb(118,91,254)");mycolorscale.push("rgb(120,84,254)");mycolorscale.push("rgb(120,84,254)");mycolorscale.push("rgb(122,77,254)");mycolorscale.push("rgb(122,77,254)");mycolorscale.push("rgb(124,70,254)");mycolorscale.push("rgb(124,70,254)");mycolorscale.push("rgb(126,64,254)");mycolorscale.push("rgb(126,64,254)");mycolorscale.push("rgb(129,57,254)");mycolorscale.push("rgb(129,57,254)");mycolorscale.push("rgb(131,50,254)");mycolorscale.push("rgb(131,50,254)");mycolorscale.push("rgb(133,43,254)");mycolorscale.push("rgb(133,43,254)");mycolorscale.push("rgb(135,36,254)");mycolorscale.push("rgb(135,36,254)");mycolorscale.push("rgb(137,29,254)");mycolorscale.push("rgb(137,29,254)");mycolorscale.push("rgb(139,22,254)");mycolorscale.push("rgb(139,22,254)");mycolorscale.push("rgb(141,15,254)");mycolorscale.push("rgb(141,15,254)");mycolorscale.push("rgb(143,8,254)");mycolorscale.push("rgb(143,8,254)");mycolorscale.push("rgb(146,1,254)");mycolorscale.push("rgb(146,1,254)");mycolorscale.push("rgb(148,1,251)");mycolorscale.push("rgb(148,1,251)");mycolorscale.push("rgb(151,1,249)");mycolorscale.push("rgb(151,1,249)");mycolorscale.push("rgb(154,1,247)");mycolorscale.push("rgb(154,1,247)");mycolorscale.push("rgb(157,1,245)");mycolorscale.push("rgb(157,1,245)");mycolorscale.push("rgb(160,1,243)");mycolorscale.push("rgb(160,1,243)");mycolorscale.push("rgb(163,1,241)");mycolorscale.push("rgb(163,1,241)");mycolorscale.push("rgb(166,1,239)");mycolorscale.push("rgb(166,1,239)");mycolorscale.push("rgb(169,1,237)");mycolorscale.push("rgb(169,1,237)");mycolorscale.push("rgb(172,1,234)");mycolorscale.push("rgb(172,1,234)");mycolorscale.push("rgb(175,1,232)");mycolorscale.push("rgb(175,1,232)");mycolorscale.push("rgb(177,1,230)");mycolorscale.push("rgb(177,1,230)");mycolorscale.push("rgb(180,1,228)");mycolorscale.push("rgb(180,1,228)");mycolorscale.push("rgb(183,1,226)");mycolorscale.push("rgb(183,1,226)");mycolorscale.push("rgb(186,1,224)");mycolorscale.push("rgb(186,1,224)");mycolorscale.push("rgb(189,1,222)");mycolorscale.push("rgb(189,1,222)");mycolorscale.push("rgb(175,2,220)");mycolorscale.push("rgb(175,2,220)");mycolorscale.push("rgb(177,7,215)");mycolorscale.push("rgb(177,7,215)");mycolorscale.push("rgb(180,12,210)");mycolorscale.push("rgb(180,12,210)");mycolorscale.push("rgb(183,18,205)");mycolorscale.push("rgb(183,18,205)");mycolorscale.push("rgb(186,23,201)");mycolorscale.push("rgb(186,23,201)");mycolorscale.push("rgb(189,28,196)");mycolorscale.push("rgb(189,28,196)");mycolorscale.push("rgb(191,34,191)");mycolorscale.push("rgb(191,34,191)");mycolorscale.push("rgb(194,39,188)");mycolorscale.push("rgb(194,39,188)");mycolorscale.push("rgb(197,45,183)");mycolorscale.push("rgb(197,45,183)");mycolorscale.push("rgb(200,50,178)");mycolorscale.push("rgb(200,50,178)");mycolorscale.push("rgb(203,55,174)");mycolorscale.push("rgb(203,55,174)");mycolorscale.push("rgb(206,61,169)");mycolorscale.push("rgb(206,61,169)");mycolorscale.push("rgb(209,65,164)");mycolorscale.push("rgb(209,65,164)");mycolorscale.push("rgb(212,70,160)");mycolorscale.push("rgb(212,70,160)");mycolorscale.push("rgb(215,76,155)");mycolorscale.push("rgb(215,76,155)");mycolorscale.push("rgb(218,81,150)");mycolorscale.push("rgb(218,81,150)");mycolorscale.push("rgb(221,87,146)");mycolorscale.push("rgb(221,87,146)");mycolorscale.push("rgb(221,88,143)");mycolorscale.push("rgb(221,88,143)");mycolorscale.push("rgb(222,90,140)");mycolorscale.push("rgb(222,90,140)");mycolorscale.push("rgb(223,91,138)");mycolorscale.push("rgb(223,91,138)");mycolorscale.push("rgb(224,93,135)");mycolorscale.push("rgb(224,93,135)");mycolorscale.push("rgb(225,95,132)");mycolorscale.push("rgb(225,95,132)");mycolorscale.push("rgb(226,96,130)");mycolorscale.push("rgb(226,96,130)");mycolorscale.push("rgb(227,98,127)");mycolorscale.push("rgb(227,98,127)");mycolorscale.push("rgb(228,100,125)");mycolorscale.push("rgb(228,100,125)");mycolorscale.push("rgb(229,101,122)");mycolorscale.push("rgb(229,101,122)");mycolorscale.push("rgb(230,103,119)");mycolorscale.push("rgb(230,103,119)");mycolorscale.push("rgb(231,104,117)");mycolorscale.push("rgb(231,104,117)");mycolorscale.push("rgb(232,106,114)");mycolorscale.push("rgb(232,106,114)");mycolorscale.push("rgb(233,108,111)");mycolorscale.push("rgb(233,108,111)");mycolorscale.push("rgb(234,109,109)");mycolorscale.push("rgb(234,109,109)");mycolorscale.push("rgb(235,111,106)");mycolorscale.push("rgb(235,111,106)");mycolorscale.push("rgb(236,113,104)");mycolorscale.push("rgb(236,113,104)");mycolorscale.push("rgb(237,116,97)");mycolorscale.push("rgb(237,116,97)");mycolorscale.push("rgb(238,119,91)");mycolorscale.push("rgb(238,119,91)");mycolorscale.push("rgb(239,122,84)");mycolorscale.push("rgb(239,122,84)");mycolorscale.push("rgb(240,126,78)");mycolorscale.push("rgb(240,126,78)");mycolorscale.push("rgb(241,129,71)");mycolorscale.push("rgb(241,129,71)");mycolorscale.push("rgb(242,132,65)");mycolorscale.push("rgb(242,132,65)");mycolorscale.push("rgb(243,136,59)");mycolorscale.push("rgb(243,136,59)");mycolorscale.push("rgb(245,139,53)");mycolorscale.push("rgb(245,139,53)");mycolorscale.push("rgb(246,142,46)");mycolorscale.push("rgb(246,142,46)");mycolorscale.push("rgb(247,146,40)");mycolorscale.push("rgb(247,146,40)");mycolorscale.push("rgb(248,149,33)");mycolorscale.push("rgb(248,149,33)");mycolorscale.push("rgb(249,152,27)");mycolorscale.push("rgb(249,152,27)");mycolorscale.push("rgb(250,156,20)");mycolorscale.push("rgb(250,156,20)");mycolorscale.push("rgb(251,159,14)");mycolorscale.push("rgb(251,159,14)");mycolorscale.push("rgb(252,162,7)");mycolorscale.push("rgb(252,162,7)");mycolorscale.push("rgb(254,166,1)");mycolorscale.push("rgb(254,166,1)");mycolorscale.push("rgb(254,170,1)");mycolorscale.push("rgb(254,170,1)");mycolorscale.push("rgb(254,174,1)");mycolorscale.push("rgb(254,174,1)");mycolorscale.push("rgb(254,178,1)");mycolorscale.push("rgb(254,178,1)");mycolorscale.push("rgb(254,182,1)");mycolorscale.push("rgb(254,182,1)");mycolorscale.push("rgb(254,186,1)");mycolorscale.push("rgb(254,186,1)");mycolorscale.push("rgb(254,191,1)");mycolorscale.push("rgb(254,191,1)");mycolorscale.push("rgb(254,194,1)");mycolorscale.push("rgb(254,194,1)");mycolorscale.push("rgb(254,198,1)");mycolorscale.push("rgb(254,198,1)");mycolorscale.push("rgb(254,202,1)");mycolorscale.push("rgb(254,202,1)");mycolorscale.push("rgb(254,206,1)");mycolorscale.push("rgb(254,206,1)");mycolorscale.push("rgb(254,211,1)");mycolorscale.push("rgb(254,211,1)");mycolorscale.push("rgb(254,215,1)");mycolorscale.push("rgb(254,215,1)");mycolorscale.push("rgb(254,219,1)");mycolorscale.push("rgb(254,219,1)");mycolorscale.push("rgb(254,223,1)");mycolorscale.push("rgb(254,223,1)");mycolorscale.push("rgb(254,227,1)");mycolorscale.push("rgb(254,227,1)");mycolorscale.push("rgb(254,212,1)");mycolorscale.push("rgb(254,212,1)");mycolorscale.push("rgb(254,214,12)");mycolorscale.push("rgb(254,214,12)");mycolorscale.push("rgb(254,216,23)");mycolorscale.push("rgb(254,216,23)");mycolorscale.push("rgb(254,218,34)");mycolorscale.push("rgb(254,218,34)");mycolorscale.push("rgb(254,220,46)");mycolorscale.push("rgb(254,220,46)");mycolorscale.push("rgb(254,222,57)");mycolorscale.push("rgb(254,222,57)");mycolorscale.push("rgb(254,224,67)");mycolorscale.push("rgb(254,224,67)");mycolorscale.push("rgb(254,226,79)");mycolorscale.push("rgb(254,226,79)");mycolorscale.push("rgb(254,228,90)");mycolorscale.push("rgb(254,228,90)");mycolorscale.push("rgb(254,230,101)");mycolorscale.push("rgb(254,230,101)");mycolorscale.push("rgb(254,232,113)");mycolorscale.push("rgb(254,232,113)");mycolorscale.push("rgb(254,234,124)");mycolorscale.push("rgb(254,234,124)");mycolorscale.push("rgb(254,236,135)");mycolorscale.push("rgb(254,236,135)");mycolorscale.push("rgb(254,238,147)");mycolorscale.push("rgb(254,238,147)");mycolorscale.push("rgb(254,240,158)");mycolorscale.push("rgb(254,240,158)");mycolorscale.push("rgb(254,242,169)");mycolorscale.push("rgb(254,242,169)");mycolorscale.push("rgb(254,245,148)");mycolorscale.push("rgb(254,245,148)");mycolorscale.push("rgb(254,245,151)");mycolorscale.push("rgb(254,245,151)");mycolorscale.push("rgb(254,246,155)");mycolorscale.push("rgb(254,246,155)");mycolorscale.push("rgb(254,246,159)");mycolorscale.push("rgb(254,246,159)");mycolorscale.push("rgb(254,247,163)");mycolorscale.push("rgb(254,247,163)");mycolorscale.push("rgb(254,247,167)");mycolorscale.push("rgb(254,247,167)");mycolorscale.push("rgb(254,248,171)");mycolorscale.push("rgb(254,248,171)");mycolorscale.push("rgb(254,248,175)");mycolorscale.push("rgb(254,248,175)");mycolorscale.push("rgb(254,249,179)");mycolorscale.push("rgb(254,249,179)");mycolorscale.push("rgb(254,250,182)");mycolorscale.push("rgb(254,250,182)");mycolorscale.push("rgb(254,250,186)");mycolorscale.push("rgb(254,250,186)");mycolorscale.push("rgb(254,251,190)");mycolorscale.push("rgb(254,251,190)");mycolorscale.push("rgb(254,251,193)");mycolorscale.push("rgb(254,251,193)");mycolorscale.push("rgb(254,252,197)");mycolorscale.push("rgb(254,252,197)");mycolorscale.push("rgb(254,252,201)");mycolorscale.push("rgb(254,252,201)");mycolorscale.push("rgb(254,253,205)");mycolorscale.push("rgb(254,253,205)");mycolorscale.push("rgb(254,254,209)");mycolorscale.push("rgb(254,254,209)");mycolorscale.push("rgb(254,254,211)");mycolorscale.push("rgb(254,254,211)");mycolorscale.push("rgb(254,254,214)");mycolorscale.push("rgb(254,254,214)");mycolorscale.push("rgb(254,254,217)");mycolorscale.push("rgb(254,254,217)");mycolorscale.push("rgb(254,254,220)");mycolorscale.push("rgb(254,254,220)");mycolorscale.push("rgb(254,254,223)");mycolorscale.push("rgb(254,254,223)");mycolorscale.push("rgb(254,254,225)");mycolorscale.push("rgb(254,254,225)");mycolorscale.push("rgb(254,254,228)");mycolorscale.push("rgb(254,254,228)");mycolorscale.push("rgb(254,254,231)");mycolorscale.push("rgb(254,254,231)");mycolorscale.push("rgb(254,254,234)");mycolorscale.push("rgb(254,254,234)");mycolorscale.push("rgb(254,254,237)");mycolorscale.push("rgb(254,254,237)");mycolorscale.push("rgb(254,254,239)");mycolorscale.push("rgb(254,254,239)");mycolorscale.push("rgb(254,254,242)");mycolorscale.push("rgb(254,254,242)");mycolorscale.push("rgb(254,254,245)");mycolorscale.push("rgb(254,254,245)");mycolorscale.push("rgb(254,254,248)");mycolorscale.push("rgb(254,254,248)");mycolorscale.push("rgb(254,254,251)");mycolorscale.push("rgb(254,254,251)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");mycolorscale.push("rgb(254,254,254)");


			var cMinZ=1e8;
			var cMaxZ=-1e8;
			var iMin=1e8;
			var iMax=-1e8;

			var mycolors=[]
			verts2.forEach(function(d){
				var length=(-d[2])/dz;
				var i = _.size(mycolorscale)-Math.round(length * _.size(mycolorscale));
				if(i<0) i=0;
				if(i>_.size(mycolorscale)-1) i=_.size(mycolorscale)-1;
				iMin=Math.min(i,iMin);
				iMax=Math.max(i,iMax);
				cMinZ=Math.min(cMinZ,length);
				cMaxZ=Math.max(cMaxZ,length);
				/*
				var r = Math.round(Math.sin(0.024 * i + 0) * 127 + 128);
				var g = Math.round(Math.sin(0.024 * i + 2) * 127 + 128);
				var b = Math.round(Math.sin(0.024 * i + 4) * 127 + 128);
				var h="rgb("+r+","+g+","+b+")"
				mycolors.push(h);
				*/
				mycolors.push(mycolorscale[i]);
			})

			console.log("surf:  ",cMinZ,cMaxZ,iMin,iMax);

	        var geom = new THREE.Geometry();
			verts2.forEach(function(d){
				geom.vertices.push(
					new THREE.Vector3(d[0],d[1],d[2])
				)
			})

			var myColorss=[];
			mycolors.forEach(function(d){
				myColorss.push(new THREE.Color(d));
			})
			/*
			myColorss.push(new THREE.Color(0xFF0000));
			myColorss.push(new THREE.Color(0xFFFF00));
			myColorss.push(new THREE.Color(0xFFFFFF));
			myColorss.push(new THREE.Color(0xFF00FF));
			*/
			tris.forEach(function(d){
				var face=new THREE.Face3(d[0],d[1],d[2]);
				if(ii!=6){
					face.vertexColors[0]=myColorss[d[0]];
					face.vertexColors[1]=myColorss[d[1]];
					face.vertexColors[2]=myColorss[d[2]];
				} else {
					face.vertexColors[0]=new THREE.Color("rgb(100,100,100)");
					face.vertexColors[1]=new THREE.Color("rgb(100,100,100)");
					face.vertexColors[2]=new THREE.Color("rgb(100,100,100)");
				}
				geom.faces.push(face)
			})

			/*
			geom.colors.push(new THREE.Color(0xFF0000));
			geom.colors.push(new THREE.Color(0xFFFF00));
			geom.colors.push(new THREE.Color(0xFFFFFF));
			geom.colors.push(new THREE.Color(0xFF00FF));
			*/
			//geom.computeFaceNormals();


			console.log("did geom");
			//var material = new THREE.MeshLambertMaterial({color: 0xff00ff,transparent:false});//,side:THREE.DoubleSide});
			//var material = new THREE.MeshBasicMaterial({color: 0xff0000, transparent: false, opacity: 0.5});

			var material = new THREE.MeshStandardMaterial({vertexColors: THREE.VertexColors,transparent:false,side:THREE.DoubleSide,flatShading:true, metalness:0,roughness:1});
			if(ii==6){
				material.transparent=true;
				material.opacity=.5;
				var localPlane = new THREE.Plane( new THREE.Vector3( 0, 0,-1 ), 0. );
				material.clippingPlanes= [ localPlane ];
			}
			//var material = new THREE.MeshBasicMaterial({ vertexColors: THREE.VertexColors });

			console.log("did material");
	        var mesh = new THREE.Mesh(geom,material)
	        //meshes.push(mesh)
			group.add(mesh)
		}

		console.log("globalMinZ:  ",globalMinZ, globalMaxZ, globalMaxZ-globalMinZ);


		window.addEventListener( 'resize', onWindowResize, false );
	}

	function onWindowResize() {
		camera.aspect = window.innerWidth / window.innerHeight;
		camera.updateProjectionMatrix();
		renderer.setSize( window.innerWidth, window.innerHeight );
	}
	function animate() {
		requestAnimationFrame( animate );
		//group.rotation.y += 0.005;
		render();
	}
	function render() {
		renderer.render( scene, camera );
	}

    
})();

<IPython.core.display.Javascript object>

In [8]:
from IPython.core.display import display, HTML
display(HTML(r"""<style id=hide>div.input{display:none;}</style><button type="button"onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">Show inputs</button>"""))